In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
import pickle

In [15]:
# df = pd.read_csv("hf://datasets/rohan-chandrashekar/Financial_Fraud_Detection/New_Dataset.csv")

In [16]:
# df.to_csv('dataset.csv')

In [17]:
df = pd.read_csv('dataset.csv')
df = df.drop('Unnamed: 0',axis=1)

In [18]:
df.head()

,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isUnauthorizedOverdraft,action__CASH_IN,action__CASH_OUT,action__DEBIT,action__PAYMENT,action__TRANSFER
0,89.24,0,89.24,0.0,0,0.0,89.24,1,0,0,0,0,0,1
1,89.24,1,89.24,0.0,1,0.0,0.00,1,0,0,1,0,0,0
2,70.62,2,70.62,0.0,2,0.0,70.62,1,0,0,0,0,0,1
3,70.62,3,70.62,0.0,3,0.0,0.00,1,0,0,1,0,0,0
4,22695.14,4,22695.14,0.0,4,0.0,22695.14,1,0,0,0,0,0,1


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3713576 entries, 0 to 3713575
Data columns (total 14 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   amount                   float64
 1   nameOrig                 int64  
 2   oldBalanceOrig           float64
 3   newBalanceOrig           float64
 4   nameDest                 int64  
 5   oldBalanceDest           float64
 6   newBalanceDest           float64
 7   isFraud                  int64  
 8   isUnauthorizedOverdraft  int64  
 9   action__CASH_IN          int64  
 10  action__CASH_OUT         int64  
 11  action__DEBIT            int64  
 12  action__PAYMENT          int64  
 13  action__TRANSFER         int64  
dtypes: float64(5), int64(9)
memory usage: 396.7 MB


In [20]:
df.describe()

,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isUnauthorizedOverdraft,action__CASH_IN,action__CASH_OUT,action__DEBIT,action__PAYMENT,action__TRANSFER
count,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06,3.713576e+06
mean,2.544929e+05,1.598620e+04,2.238621e+06,2.156862e+06,2.854496e+04,6.405038e+04,1.499370e+05,9.069102e-02,1.098591e-02,6.003340e-01,1.721850e-01,2.634065e-02,1.255596e-01,7.558079e-02
std,5.702028e+05,2.396252e+04,1.470512e+06,1.522028e+06,2.988465e+04,2.071222e+05,4.868473e+05,2.871693e-01,1.042364e-01,4.898297e-01,3.775412e-01,1.601463e-01,3.313524e-01,2.643262e-01
min,0.000000e+00,0.000000e+00,-1.994681e+05,-1.994681e+05,0.000000e+00,-1.142105e+05,-2.316637e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.870973e+04,4.665000e+03,8.775562e+05,6.710188e+05,1.306100e+04,8.740160e+03,1.685267e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.231877e+05,1.108400e+04,2.380429e+06,2.316763e+06,2.408800e+04,3.931027e+04,4.867997e+04,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.993595e+05,1.804300e+04,3.397556e+06,3.376386e+06,3.443600e+04,7.737530e+04,9.067437e+04,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.191388e+07,1.884540e+05,3.348603e+07,3.370815e+07,2.191040e+05,8.522232e+06,2.191388e+07,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [21]:
df.isnull().sum()

amount                     0
nameOrig                   0
oldBalanceOrig             0
newBalanceOrig             0
nameDest                   0
oldBalanceDest             0
newBalanceDest             0
isFraud                    0
isUnauthorizedOverdraft    0
action__CASH_IN            0
action__CASH_OUT           0
action__DEBIT              0
action__PAYMENT            0
action__TRANSFER           0
dtype: int64

In [22]:
X = df.drop('isFraud',axis=1)
y = df['isFraud']

In [23]:
X_train , X_valid , y_train , y_valid = train_test_split(X,y,test_size=0.3,random_state=39)

In [24]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_valid = scale.transform(X_valid)

In [25]:
lightgbm = LGBMClassifier()
lightgbm.fit(X_train,y_train)
y_pred = lightgbm.predict(X_valid)
print(classification_report(y_valid,y_pred))

[LightGBM] [Info] Number of positive: 235978, number of negative: 2363525
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054837 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1803
[LightGBM] [Info] Number of data points in the train set: 2599503, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.090778 -> initscore=-2.304171
[LightGBM] [Info] Start training from score -2.304171
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1013263
           1       1.00      1.00      1.00    100810

    accuracy                           1.00   1114073
   macro avg       1.00      1.00      1.00   1114073
weighted avg       1.00      1.00      1.00   1114073



In [26]:
with open('model.pkl','wb') as f:
    pickle.dump(lightgbm,f)